## Predator Prey Ecosystem

## The problem statement solved by the given code is to simulate the population dynamics of a predator-prey ecosystem using the Lotka-Volterra equations. The Lotka-Volterra equations are a pair of first-order nonlinear differential equations that describe the interactions between two species in a closed ecosystem, where one species (the predator) feeds on the other (the prey).

The code initializes the initial populations of prey and predator and creates SYCL buffers to store the populations. Then, it submits a SYCL kernel using a parallel_for loop that calculates the populations at each time step using the Lotka-Volterra equations. The SYCL kernel ensures that the populations remain non-negative. After the simulation is complete, the final populations of prey and predator are printed to the console.

The Lotka-Volterra equations used in the code are:

dP/dt = aP - bPQ
dQ/dt = -cQ + dPQ

where P is the population of prey, Q is the population of predator, a is the natural growth rate of prey in the absence of predation, b is the rate at which predators consume prey, c is the natural death rate of predators in the absence of food, and d is the efficiency of turning prey into predator offspring.

In [1]:
%%writefile lab/simple.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <CL/sycl.hpp>
#include <iostream>
#include <vector>

using namespace cl::sycl;

const float a = 1.0f;
const float b = 0.1f;
const float c = 0.01f;
const float d = 1.5f;

int main() {
    const float dt = 0.01f; // time step
    const int N = 1000; // number of time steps

    std::vector<float> prey(N);
    std::vector<float> predator(N);

    prey[0] = 10.0f;
    predator[0] = 5.0f;

    // Create buffers
    buffer<float, 1> prey_buf(prey.data(), range<1>(N));
    buffer<float, 1> predator_buf(predator.data(), range<1>(N));

    queue q;

    q.submit([&](handler& cgh) {
        auto prey_acc = prey_buf.get_access<access::mode::read_write>(cgh);
        auto predator_acc = predator_buf.get_access<access::mode::read_write>(cgh);

        cgh.parallel_for<class simulation_kernel>(range<1>(N-1), [=](id<1> i) {
            // Lotka-Volterra equations
            prey_acc[i+1] = prey_acc[i] + dt * (a * prey_acc[i] - b * prey_acc[i] * predator_acc[i]);
            predator_acc[i+1] = predator_acc[i] + dt * (-c * predator_acc[i] + d * prey_acc[i] * predator_acc[i]);

            // Ensure non-negative populations
            if (prey_acc[i+1] < 0.0f) {
                prey_acc[i+1] = 0.0f;
            }
            if (predator_acc[i+1] < 0.0f) {
                predator_acc[i+1] = 0.0f;
            }
        });
    });

    q.wait();

    std::cout << "Final prey population: " << prey_buf.get_access<access::mode::read>()[N-1] << std::endl;
    std::cout << "Final predator population: " << predator_buf.get_access<access::mode::read>()[N-1] << std::endl;

    return 0;
}



Overwriting lab/simple.cpp


In [3]:
! chmod 755 q; chmod 755 run_simple.sh;if [ -x "$(command -v qsub)" ]; then ./q run_simple.sh; else ./run_simple.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2260557.v-qsvr-1           ...ub-singleuser u182761         00:00:08 R jupyterhub     
2260574.v-qsvr-1           run_simple.sh    u182761         00:00:00 R batch          
2260577.v-qsvr-1           run_simple.sh    u182761                0 Q batch          

Waiting for Output ███████████ Done⬇

########################################################################
#      Date:           Wed 22 Mar 2023 11:41:21 PM PDT
#    Job ID:           2260577.v-qsvr-1.aidevcloud
#      User:           u182761
# Resources:           cput=75:00:00,neednodes=1:gpu:ppn=2,nodes=1:gpu:ppn=2,walltime=06:00:00
########################################################################

## u182761 is compiling DPCPP_Essentials Module1 -- oneAPI Intro sample - 1 of 1 simple.cpp
Final pre